In [1]:
import sympy
import math

In [2]:
# Σtの位置と姿勢行列を定義
xt, yt, zt = sympy.symbols('x_t y_t z_t')
r11, r12, r13 = sympy.symbols('r11 r12 r13')
r21, r22, r23 = sympy.symbols('r21 r22 r23')
r31, r32, r33 = sympy.symbols('r31 r32 r33')
# U0t: Σ0からΣtへの目標位置姿勢の同次変換行列(逆運動学の入力）
U0t = sympy.Matrix([[r11, r12, r13, xt], [r21, r22, r23, yt], [r31, r32, r33, zt], [0, 0, 0, 1]])
U0t

Matrix([
[r11, r12, r13, x_t],
[r21, r22, r23, y_t],
[r31, r32, r33, z_t],
[  0,   0,   0,   1]])

In [3]:
# 順運動学のための関節角度の定義
theta1, theta2, theta3, theta4, theta5, theta6 =sympy.symbols('theta1 theta2 theta3 theta4 theta5 theta6')
# 短く書くためにSin,Cosも定義しておく
c1, c2, c3, c4, c5, c6 = sympy.cos(theta1), sympy.cos(theta2), sympy.cos(theta3), sympy.cos(theta4), sympy.cos(theta5), sympy.cos(theta6)
s1, s2, s3, s4, s5, s6 = sympy.sin(theta1), sympy.sin(theta2), sympy.sin(theta3), sympy.sin(theta4), sympy.sin(theta5), sympy.sin(theta6)
# d1,d2,d3,d4,d5: 各リンク長のパラメータ（ロボットの自由度配置の図を参照）
d1, d2, d3 =sympy.symbols('d1 d2 d3')
a = sympy.symbols('a')

In [4]:
# 順運動学の計算のための各関節の同時変換行列
T01 = sympy.Matrix([[c1, -s1, 0, 0], [s1, c1, 0, 0], [0, 0, 1, d1], [0, 0, 0, 1]])
T12 = sympy.Matrix([[c2, -s2, 0, 0], [0, 0, 1, 0], [-s2, -c2, 0, 0], [0, 0, 0, 1]])
T23 = sympy.Matrix([[c3, -s3, 0, 0], [0, 0, -1, -d2], [s3, c3, 0, 0], [0, 0, 0, 1]])

T34 = sympy.Matrix([[c4, -s4, 0, a], [0, 0, -1, 0], [s4, c4, 0, 0], [0, 0, 0, 1]])

T45 = sympy.Matrix([[c5, -s5, 0, -a], [0, 0, 1, d3], [-s5, -c5, 0, 0], [0, 0, 0, 1]])

#T56 = sympy.Matrix([[c6, -s6, 0, 0], [0, 0, -1, 0], [s6, c6, 0, 0], [0, 0, 0, 1]])
T6t = sympy.Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
# Σ0からΣtへの順運動学
T0t=T01*T12*T23*T34*T45*T6t
T0t

Matrix([
[((-sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3))*cos(theta4) + sin(theta2)*sin(theta4)*cos(theta1))*cos(theta5) - (sin(theta1)*cos(theta3) + sin(theta3)*cos(theta1)*cos(theta2))*sin(theta5), -((-sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3))*cos(theta4) + sin(theta2)*sin(theta4)*cos(theta1))*sin(theta5) - (sin(theta1)*cos(theta3) + sin(theta3)*cos(theta1)*cos(theta2))*cos(theta5), -(-sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3))*sin(theta4) + sin(theta2)*cos(theta1)*cos(theta4), -a*((-sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3))*cos(theta4) + sin(theta2)*sin(theta4)*cos(theta1)) + a*(-sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3)) + d2*sin(theta2)*cos(theta1) + d3*(-(-sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3))*sin(theta4) + sin(theta2)*cos(theta1)*cos(theta4))],
[ ((sin(theta1)*cos(theta2)*cos(theta3) + sin(theta3)*cos(theta1))*cos(theta4) + sin(theta1)*sin(theta2)*sin(thet

In [5]:
# pt: 順運動学から計算したΣtの位置（Panda初期姿勢）
pt = sympy.Matrix([T0t[0,3], T0t[1,3], T0t[2,3]])
#angle = 0.3
# Pandaのリンクパラメータを設定
param = [(theta1, 1.5386),(theta2, -0.7350002),(theta3, 6.98e-09), (theta4, -2.205001), (theta5, -6.20E-09),
         (d1, 0.333), (d2, 0.316), (d3, 0.384), (a, 0.0825)]
# ROSのURDFから計算した順運動学と比較して、正しいことを確認する
check_tool = T0t.subs(param)
print('s1..5=', s1.subs(param), s2.subs(param), s3.subs(param), s4.subs(param), s5.subs(param))
print('c1..5=', c1.subs(param), c2.subs(param), c3.subs(param), c4.subs(param), c5.subs(param))
# このcheck_toolは後ほど逆運動学の正しいことの確認にも使う
# X=0.223070384, Y=0.743498401, Z=0.007166766 (Unity) ===> Y, Z, X
# X=0.007166766, Y=0.223070384, Z=0.743498401 (AGX)
check_tool

s1..5= 0.999481743041692 -0.670587304004041 6.98000000000000e-9 -0.805543211773116 -6.20000000000000e-9
c1..5= 0.0321907646125725 0.741830619284882 1.00000000000000 -0.592537031725658 1.00000000000000


Matrix([
[0.00323920400607653,   -0.999481743188292, 0.0320273725225944, 0.00717998157330358],
[  0.100572787540687,   0.0321907600608295,  0.994408803949865,   0.222929267460331],
[ -0.994924430277849, -1.08492308496709e-8,  0.100624937447419,   0.743463169752088],
[                  0,                    0,                  0,                   1]])

In [6]:
pt
##### Panda　はここまで #####

Matrix([
[-a*((-sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3))*cos(theta4) + sin(theta2)*sin(theta4)*cos(theta1)) + a*(-sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3)) + d2*sin(theta2)*cos(theta1) + d3*(-(-sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3))*sin(theta4) + sin(theta2)*cos(theta1)*cos(theta4))],
[   -a*((sin(theta1)*cos(theta2)*cos(theta3) + sin(theta3)*cos(theta1))*cos(theta4) + sin(theta1)*sin(theta2)*sin(theta4)) + a*(sin(theta1)*cos(theta2)*cos(theta3) + sin(theta3)*cos(theta1)) + d2*sin(theta1)*sin(theta2) + d3*(-(sin(theta1)*cos(theta2)*cos(theta3) + sin(theta3)*cos(theta1))*sin(theta4) + sin(theta1)*sin(theta2)*cos(theta4))],
[                                                                                                                                                          -a*(-sin(theta2)*cos(theta3)*cos(theta4) + sin(theta4)*cos(theta2)) - a*sin(theta2)*cos(theta3) + d1 + d2*cos(theta2) + d3*(sin(theta2)*sin(theta4)*

In [7]:
xt, yt, zt = pt
display(xt)
print()
display(yt)
print()
display(zt)

-a*((-sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3))*cos(theta4) + sin(theta2)*sin(theta4)*cos(theta1)) + a*(-sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3)) + d2*sin(theta2)*cos(theta1) + d3*(-(-sin(theta1)*sin(theta3) + cos(theta1)*cos(theta2)*cos(theta3))*sin(theta4) + sin(theta2)*cos(theta1)*cos(theta4))

-a*((sin(theta1)*cos(theta2)*cos(theta3) + sin(theta3)*cos(theta1))*cos(theta4) + sin(theta1)*sin(theta2)*sin(theta4)) + a*(sin(theta1)*cos(theta2)*cos(theta3) + sin(theta3)*cos(theta1)) + d2*sin(theta1)*sin(theta2) + d3*(-(sin(theta1)*cos(theta2)*cos(theta3) + sin(theta3)*cos(theta1))*sin(theta4) + sin(theta1)*sin(theta2)*cos(theta4))

-a*(-sin(theta2)*cos(theta3)*cos(theta4) + sin(theta4)*cos(theta2)) - a*sin(theta2)*cos(theta3) + d1 + d2*cos(theta2) + d3*(sin(theta2)*sin(theta4)*cos(theta3) + cos(theta2)*cos(theta4))

In [8]:
dxt2 = xt.diff(theta2).simplify()
dxt3 = xt.diff(theta3).simplify()
dxt4 = xt.diff(theta4).simplify()
dxt5 = xt.diff(theta5).simplify()
display(dxt2)
print()
display(dxt3)
print()
display(dxt4)
print()
display(dxt5)

(a*(sin(theta2)*cos(theta3)*cos(theta4) - sin(theta4)*cos(theta2)) - a*sin(theta2)*cos(theta3) + d2*cos(theta2) + d3*(sin(theta2)*sin(theta4)*cos(theta3) + cos(theta2)*cos(theta4)))*cos(theta1)

(sin(theta1)*cos(theta3) + sin(theta3)*cos(theta1)*cos(theta2))*(a*cos(theta4) - a + d3*sin(theta4))

-a*((sin(theta1)*sin(theta3) - cos(theta1)*cos(theta2)*cos(theta3))*sin(theta4) + sin(theta2)*cos(theta1)*cos(theta4)) + d3*((sin(theta1)*sin(theta3) - cos(theta1)*cos(theta2)*cos(theta3))*cos(theta4) - sin(theta2)*sin(theta4)*cos(theta1))

0

In [9]:
dyt2 = yt.diff(theta2).simplify()
dyt3 = yt.diff(theta3).simplify()
dyt4 = yt.diff(theta4).simplify()
dyt5 = yt.diff(theta5).simplify()
display(dyt2)
print()
display(dyt3)
print()
display(dyt4)
print()
display(dyt5)

(a*(sin(theta2)*cos(theta3)*cos(theta4) - sin(theta4)*cos(theta2)) - a*sin(theta2)*cos(theta3) + d2*cos(theta2) + d3*(sin(theta2)*sin(theta4)*cos(theta3) + cos(theta2)*cos(theta4)))*sin(theta1)

(sin(theta1)*sin(theta3)*cos(theta2) - cos(theta1)*cos(theta3))*(a*cos(theta4) - a + d3*sin(theta4))

a*((sin(theta1)*cos(theta2)*cos(theta3) + sin(theta3)*cos(theta1))*sin(theta4) - sin(theta1)*sin(theta2)*cos(theta4)) - d3*((sin(theta1)*cos(theta2)*cos(theta3) + sin(theta3)*cos(theta1))*cos(theta4) + sin(theta1)*sin(theta2)*sin(theta4))

0

In [10]:
dzt2 = zt.diff(theta2).simplify()
dzt3 = zt.diff(theta3).simplify()
dzt4 = zt.diff(theta4).simplify()
display(dzt2)
print()
display(dzt3)
print()
display(dzt4)

a*(sin(theta2)*sin(theta4) + cos(theta2)*cos(theta3)*cos(theta4)) - a*cos(theta2)*cos(theta3) - d2*sin(theta2) - d3*(sin(theta2)*cos(theta4) - sin(theta4)*cos(theta2)*cos(theta3))

(-a*cos(theta4) + a - d3*sin(theta4))*sin(theta2)*sin(theta3)

-a*(sin(theta2)*sin(theta4)*cos(theta3) + cos(theta2)*cos(theta4)) + d3*(sin(theta2)*cos(theta3)*cos(theta4) - sin(theta4)*cos(theta2))

In [11]:
dX2 = dxt2.subs(param)
dX3 = dxt3.subs(param)
dX4 = dxt4.subs(param)
dY2 = dyt2.subs(param)
dY3 = dyt3.subs(param)
dY4 = dyt4.subs(param)
dZ2 = dzt2.subs(param)
dZ3 = dzt3.subs(param)
dZ4 = dzt4.subs(param)
print(f'dp2 = ({dX2}, {dY2}, {dZ2})')
print(f'dp3 = ({dX3}, {dY3}, {dZ3})')
print(f'dp4 = ({dX4}, {dY4}, {dZ4})')

dp2 = (0.0132131232796198, 0.410250444358234, -0.223044861913008)
dp3 = (-0.440484495985471, 0.0141868828945311, -2.06284459133649E-9)
dp4 = (-0.00388611019188129, -0.120658676818128, 0.373749423887282)
